In [1]:
import numpy as np
import pandas as pd
import sys

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [21]:
categorical_nutrition_df = pd.read_csv('categorical_nutrition.csv')
ipaq_df = pd.read_csv('ipaq.csv')
physical_activity_df = pd.read_csv('physical_activity.csv')
sovershen_one_df = pd.read_csv('sovershen_one.csv')
sovershen_two_df = pd.read_csv('sovershen_two.csv')
labels_df = pd.read_csv('labels.csv')
pasport_df = pd.read_csv('pasport_df.csv')

In [23]:
df = pd.merge(categorical_nutrition_df.drop(columns=['Unnamed: 0']), ipaq_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df = pd.merge(df, physical_activity_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df = pd.merge(df, sovershen_one_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df = pd.merge(df, sovershen_two_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df = pd.merge(df, labels_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df = pd.merge(df, pasport_df.drop(columns=['Unnamed: 0']), on='user_id', how='left')
df.drop(columns='user_id', inplace = True)

In [72]:
type(df.loc[0,'labels'])

str

In [63]:
for l in df.loc[:,'labels']:
    print(l)

[0, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 1]
[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 1, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 1]
[0, 0, 0, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 1]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 1, 1]
[1, 0, 1, 1, 1]
[1, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 1, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 0, 0

In [24]:
#split to dataset
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1021 train examples
256 validation examples
320 test examples


In [68]:
# Вспомогательный метод для создания tf.data dataset из датафрейма Pandas
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop('labels')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [67]:
batch_size = 5 # Небольшой размер пакета используется для демонстрационных целей
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

ValueError: Value tf.Tensor(b'[1, 0, 1, 0, 0]', shape=(), dtype=string) has insufficient rank for batching.

In [29]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of age:', feature_batch['age'])
    print('A batch of labels:', label_batch )

Every feature: ['milk', 'sour_milk', 'curd', 'cheese', 'oil', 'fruits', 'dried_fruits', 'vegetables', 'legumes', 'eggs', 'meat', 'fish', 'farinaceous_food', 'cereals', 'soups', 'juices', 'tea', 'coffee', 'soda', 'water', 'alcohol', 'sweets', 'Общая физ.активность МЕТ-мин/нед_ang', 'Активность за неделю', 'Ср.значение/день', 'Ср.активность/день_работа', 'Ср.активность/день_транспорт', 'Ср.активность/день_спорт', '%_работа', '%_транспорт', '%_спорт', 'Общее время сидения в мин', 'Прогулка MET-мин/нед', 'Умеренный MET-мин/нед', 'Энергичный MET-мин/нед', 'Общая физ.активность MET-мин/нед_КФ', 'РО-Ходьба MET-мин/нед', 'РО-Умеренные MET-мин/нед', 'РО-Энергичные MET-мин/нед', 'РО-Общая продолжительность MET-мин/нед', 'АТ-Ходьба MET-мин/нед', 'АТ-Велоспорт MET-мин/нед', 'АТ-Общий транспорт MET-мин/нед', 'ДС-Энергичные MET-мин/нед', 'ДС-Умеренные двор MET-мин/нед', 'ДС-Умеренные дом MET-мин/нед', 'ДС-Общая продолжительность MET-мин/нед', 'ЗД-Ходьба MET-мин/нед', 'ЗД-Умеренные MET-мин/нед', 'ЗД-

In [30]:
# Мы используем этот пакте для демонстрации нескольких видов столбцов признаков
example_batch = next(iter(train_ds))[0]

In [31]:
# Служебный метод для создания столбца признаков
# и преобразования пакета данных
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [32]:
age = feature_column.numeric_column("age")
demo(age)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[57.]
 [36.]
 [55.]
 [59.]
 [63.]]


In [39]:
feature_columns = []

# численные столбцы
for header in list(df.columns.drop('labels')):
    feature_columns.append(feature_column.numeric_column(header))

In [44]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [46]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [65]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [51]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
array([[0],
       [0],
       [5],
       [8],
       [6],
       [0],
       [3],
       [3],
       [6],
       [2],
       [6],
       [5],
       [5],
       [5],
       [1],
       [2],
       [5],
       [2],
       [0],
       [3],
       [5],
       [5],
       [1],
       [3],
       [3],
       [0],
       [2],
       [0],
       [3],
       [0],
       [5],
       [3]])>, 'sour_milk': <tf.Tensor: shape=(32, 1), dtype=int64, numpy=
array([[4],
       [4],
       [5],
       [4],
       [3],
       [5],
       [5],
       [3],
       [4],
       [4],
       [3],
       [5],
       [5],
       [3],
       [5],
       [3],
       [6],
       [5],
       [3],
       [6],
       [5],
       [4],
       [3],
       [5],
       [5],
       [5],
       [6],
       [5],
       [3],
       [5],
       [5],
       [1]])>, 'curd': <tf.Tensor: shape=(32, 1), dtype=int64, numpy=
array([[1],
       [4],
       [5],
       [2],
       [2],
       [5],
       [3],
       [1],
     

UnimplementedError: Cast string to float is not supported [Op:Cast]